##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

In [4]:
%pip install -U -q "google-genai>=1.4.0"

Note: you may need to restart the kernel to use updated packages.


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google import genai

GOOGLE_API_KEY = input("Enter your API key here:")
client = genai.Client(api_key=GOOGLE_API_KEY)

## Select your model

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [39]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash"] {"allow-input":true, isTemplate: true}

## Run your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Give me python code to sort a list"
)

display(Markdown(response.text))

# Data

## Database Integration

In [75]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append('../../')

from Database.db import SessionLocal
from Database_Table import Inventory, Order

def getDbContent():
    session = SessionLocal()
    inventory_records = session.query(Inventory).all()
    order_records = session.query(Order).all()
    session.close()
    return inventory_records, order_records

inventory, order = getDbContent()

In [ ]:
def dbtoList(records):
    output_list = []
    for r in records:
        if isinstance(r, Inventory):
            data = {
                "ItemId": r.ItemId, 
                "Date": r.Date, 
                "Quantity": r.ItemQuantity, 
                "Category": r.ItemCategory, 
                "UnitsSold": r.UnitsSold, 
                "Weight": r.Weight, 
                "Size": r.Size, 
                "Priority": r.Priority, 
                "Dispose": r.Dispose
            }
        elif isinstance(r, Order):
            data = {
                "OrderId": r.OrderId,
                "ItemId": r.ItemId, 
                "OrderQuantity": r.OrderQuantity, 
                "Sales": r.Sales, 
                "Price": r.Price, 
                "Discount": r.Discount,
                "Profit": r.Profit, 
                "DateOrdered": r.DateOrdered,
                "DateReceived": r.DateReceived,
                "CustomerSegment": r.CustomerSegment
            }
        else:
            continue
        output_list.append(data)
    
    return output_list

inventoryData = dbtoList(inventory)
orderData = dbtoList(order)

## Supervised Models

In [32]:
import pickle
import joblib
from datetime import datetime

class Supervised_Models:
    # Location Prediction Model
    def predict_location(input_data):
        with open('../../Supervised models/Samuel/storage_prediction_model.pkl', 'rb') as file:
            storage_prediction_model = pickle.load(file)
            
        categorical_features = {
            'Priority': ['High','Low','Medium'],
            'Product_Type': ['Clothing','Electronics','Home Goods','Sports'],
            'Size': ['Large','Medium','Small']
        }
        numerical_features = ['Order_Quantity', 'Weight']
        one_hot_columns = []
        
        for feature, values in categorical_features.items():
            for value in values:
                one_hot_columns.append(f"{feature}_{value}")
            
        # Combine with numerical features to get all feature names
        all_feature_names = one_hot_columns + numerical_features

        features_dict = {col: 0 for col in all_feature_names}
    
        # Set one-hot encoded features
        for feature, values in categorical_features.items():
            if feature in input_data:
                selected_value = input_data[feature]
                one_hot_col = f"{feature}_{selected_value}"
                if one_hot_col in features_dict:
                    features_dict[one_hot_col] = 1
    
        # Set numerical features
        for feature in numerical_features:
            if feature in input_data:
                features_dict[feature] = float(input_data[feature])
        
        # Convert to array in the correct order
        features_array = np.array([features_dict[col] for col in all_feature_names]).reshape(1, -1)

        prediction = storage_prediction_model.predict(features_array)
        return prediction

    
    # Demand forecast model
    def predict_demand_forecast(input_data):
        demand_forecast_model = joblib.load('../../Supervised models/ShernFai/model/salesforecast_model.pkl')
        with open('../../Supervised models/ShernFai/model/salesforecast_preprocessor.pkl', 'rb') as f:
            preprocessor_data = pickle.load(f)

        # Extract preprocessor components
        le_category = preprocessor_data['label_encoder_category']
        reference_date = preprocessor_data['reference_date']
        unique_categories = preprocessor_data['unique_categories']
        feature_columns = preprocessor_data['feature_columns']

        print(unique_categories)

        # Get data
        category_name = input_data['category']
        future_month = input_data['month']
        product_category_id = int(input_data['product_category_id'])
        avg_price = float(input_data['avg_price'])
        customer_segment = input_data['customer_segment']
        discount_rate = float(input_data['discount_rate'])
        
        # Parse the future date
        future_date = pd.to_datetime(future_month)
        
        # Calculate time features for the future date
        months_since_start = ((future_date - reference_date).days / 30.44)
        
        # Create test data with numerical time features
        test_data = {
            'Category Name': category_name,
            'Product Category Id': product_category_id,
            'Average Product Price': avg_price,
            'Customer Segment': customer_segment,
            'Order Item Discount Rate': discount_rate,
            # Time features (numerical - can handle ANY future date!)
            'Year': future_date.year,
            'Month': future_date.month,
            'Quarter': future_date.quarter,
            'Months_Since_Start': int(months_since_start),
            'Month_Sin': np.sin(2 * np.pi * future_date.month / 12),
            'Month_Cos': np.cos(2 * np.pi * future_date.month / 12),
            'Year_Trend': future_date.year - reference_date.year
        }
        
        # Create DataFrame
        test_df = pd.DataFrame([test_data])
        
        # Handle unknown category
        if category_name not in le_category.classes_:
            print(f"Unknown category '{category_name}' - using default: {le_category.classes_[0]}")
            test_df['Category Name'] = le_category.classes_[0]
            category_name = le_category.classes_[0]
        
        # Encode category
        test_df['Category Name'] = le_category.transform(test_df['Category Name'])
        
        # One-hot encode customer segment
        test_df = pd.get_dummies(test_df, columns=['Customer Segment'], drop_first=True)
        
        # Ensure same columns as training (crucial!)
        test_df = test_df.reindex(columns=feature_columns, fill_value=0)
        
        # Make prediction
        prediction = demand_forecast_model.predict(test_df)[0]
        
        return prediction, category_name
        


# Testing
print(Supervised_Models.predict_location({
    "Priority": "High",
    "Product_Type": "Sports",
    "Size": "Small",
    "Order_Quantity": 9,
    "Weight": 9.78
}))

print(Supervised_Models.predict_demand_forecast({
    'category': "Accessories",
    'month': 5,
    'product_category_id': 2,
    'avg_price': 10.0,
    'customer_segment': "Consumer",
    'discount_rate': 0.12
}))

        

['A-5']
['Accessories', 'As Seen on  TV!', 'Baby ', 'Baseball & Softball', 'Basketball', 'Books ', 'Boxing & MMA', 'CDs ', 'Cameras ', 'Camping & Hiking', 'Cardio Equipment', "Children's Clothing", 'Cleats', 'Computers', 'Consumer Electronics', 'Crafts', 'DVDs', 'Electronics', 'Fishing', 'Fitness Accessories', 'Garden', "Girls' Apparel", 'Golf Apparel', 'Golf Bags & Carts', 'Golf Balls', 'Golf Gloves', 'Golf Shoes', 'Health and Beauty', 'Hockey', 'Hunting & Shooting', 'Indoor/Outdoor Games', "Kids' Golf Clubs", 'Lacrosse', "Men's Clothing", "Men's Footwear", "Men's Golf Clubs", 'Music', 'Pet Supplies', 'Shop By Sport', 'Soccer', 'Sporting Goods', 'Strength Training', 'Tennis & Racquet', 'Toys', 'Trade-In', 'Video Games', 'Water Sports', "Women's Apparel", "Women's Clothing", "Women's Golf Clubs"]
(25.296114, 'Accessories')


D:\Yong Meng Lee\anaconda3\envs\it3100\lib\site-packages\xgboost\core.py:158: UserWarning: [20:05:34] WARNING: C:\b\abs_52v3kadn8m\croot\xgboost-split_1748343554494\work\src\gbm\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
D:\Yong Meng Lee\anaconda3\envs\it3100\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#sec

# Sections

## Products Overview

## Category Distribution

## Product Usage Forecast

## Sales Insights

In [ ]:
sales_insights = client.models.generate_content(
    model=MODEL_ID,
    contents=
'''Generate a sales insights report that describes information for the following areas:

1. Sales Trends: Summarize sales over the past month, quarter, and year. Provide insights on which product categories are seeing the highest demand.
2. Product Performance: Analyze the best-selling products by quantity and revenue. Highlight the top 3 performing products.
3. Product Demand Forecast: Based on the historical sales and usage probability, forecast the demand for the next period.
4. Restocking or Discontinuation: Recommend which products should be restocked and which should be discontinued, based on sales trends and inventory levels.

Data:
- Historical sales data: {sales_data}
- Sales volume predictions: {sales_predictions}
- Product categories: {product_categories}
- Current inventory levels: {current_inventory}
- Usage probabilities: {usage_probabilities}
- Product classifications: {product_classifications}'''
)

display(Markdown(sales_insights.text))

## Storage Optimizations

In [ ]:
storage_optimizations = client.models.generate_content(
    model=MODEL_ID,
    contents=
'''Provide detailed storage optimization recommendations based on:

1. Current storage utilization metrics
2. Model-predicted optimal locations vs current locations
3. List of items flagged for relocation, including:
   - Current location
   - Recommended location
   - Reason for recommendation
4. Estimated space savings from optimizations

Data:
{current_storage_data}
{location_predictions}
{anomaly_flags}'''
)

display(Markdown(storage_optimizations.text))

## Anomalies Detected

In [ ]:
anomalies_detected = client.models.generate_content(
    model=MODEL_ID,
    contents=
'''Generate an anomalies section that:

1. Lists all detected anomalies categorized by:
   - Misplaced items
   - Severity
   - Data inconsistencies
2. For each anomaly, provide:
   - Item details
   - Nature of anomaly
   - Recommended corrective action
   - Priority level (high/medium/low)
3. Summary of potential impact if not addressed.

Data:
'''
)

display(Markdown(anomalies_detected.text))

## Summary

# PDF File Generation

## Set the temperature

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `types.GenerateContentConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

Note: Although you can set the `candidate_count` in the generation_config, 2.0 and later models will only return a single candidate at the this time.

In [14]:
from google.genai import types

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Give me a numbered list of cat facts.',
    config=types.GenerateContentConfig(
        max_output_tokens=2000,
        temperature=1.9,
        stop_sequences=['\n6'] # Limit to 5 facts.
    )
)

display(Markdown(response.text))

Here are some cat facts for you:

1.  Domestic cats spend about **70% of their day sleeping** and 15% grooming.
2.  The average lifespan of an outdoor cat is significantly shorter (2-5 years) compared to an **indoor cat (10-15+ years)**.
3.  Cats use their **whiskers** to "feel" the world around them, gauge openings, and detect changes in air currents. They are highly sensitive tactile organs.
4.  A group of cats is called a **clowder**, a group of kittens is called a kindle.
5.  Cats have a unique scent gland on their paws, which is why they **knead** — it's a way of marking territory and showing contentment.

## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out the [get started notebook](./Get_started.ipynb) or the [documentation's quickstart](https://ai.google.dev/tutorials/python_quickstart).